In [1]:
import torch
import sys
import pickle
import numpy as np

sys.path.append('..')
from prosim.config.default import Config, get_config
from prosim.core.registry import registry

from prosim.demo.vis import plot_full_map, plot_batch_prompts, plot_model_output, plot_demo_fig, extract_lane_vecs
from torch.utils.data import DataLoader
from torch import tensor
nan = torch.nan

#@title Load ProSim package and util functions
import torch
import sys
import pickle
import numpy as np

from prosim.config.default import Config, get_config
from prosim.core.registry import registry

/u/shuhan/anaconda3/envs/ps_online/lib/python3.8/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/u/shuhan/anaconda3/envs/ps_online/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-22 00:00:22.144683: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-22 00:00:22.287497: I tensorflow/core/util/port.cc:104] oneDNN custom op

In [2]:
config = get_config('cfg/waymo_demo.yaml', cluster='local')
dataset = registry.get_dataset("prosim_imitation")(config, 'train')

TRAIN: use all scenes
cache location: /u/shuhan/code_release/prosim/demo_dataset/trajdata_cache
do not cache map
['goal', 'v_action_tag', 'drag_point', 'llm_text_OneText']
/u/shuhan/code_release/prosim_instruct_520k/text_prompts/train/70b_0.8_0.9_action_prompt_v2
Condition: Loading all LLM text files from folder:  /u/shuhan/code_release/prosim_instruct_520k/text_prompts/train/70b_0.8_0.9_action_prompt_v2


Loading LLM text files: 0it [00:00, ?it/s]

Condition: Loaded LLM text data for 0 scenes.
Loading data for matched scene tags: ['train-waymo_train']


check cached scenes (Serially): 100%|██████████| 16/16 [00:00<00:00, 52883.27it/s]

cached: 16, uncached: 0
16 scenes in the scene index.
Loaded data index from /u/shuhan/code_release/prosim/demo_dataset/trajdata_cache/data_indexes/891a1309864c26c41a7d5f6c058765b1fe119436/data_index.dill



Structuring Scene Data Index: 100%|██████████| 16/16 [00:00<00:00, 362750.62it/s]

data index already-cached, igore save_index == True).
all
TRAIN: Uniform Sample with rate: 1, sample 16 from 16 frames


In [3]:

dataloader = DataLoader(dataset, batch_size=1, shuffle=True, collate_fn=dataset.get_collate_fn(), num_workers=0)

for batch in dataloader:
  break

DEBUG: number of condition samples in each batch: [30]
DEBUG: number of condition samples in each batch: [67]
DEBUG: number of condition samples in each batch: [30]
DEBUG: number of condition samples in each batch: [20]


### Check Language Prompts

In [4]:
text_prompts = batch.extras['condition']['llm_text_OneText']['input']
print('\n'.join(text_prompts))

Direct <A24> and <A29> to park from 40 seconds onwards.
<A21> turns right and accelerates, while <A28> accelerates and drives straight.
Most vehicles, including <A2>, <A6>, <A3>, <A1>, <A7>, <A9>, <A18>, <A13>, and <A12>, remain parked throughout the scenario.
Ensure <A26> drives straight and accelerates after 40 seconds.
<A22>, <A15>, <A10>, <A11>, <A23>, <A17>, <A19>, <A14>, and <A16> all drive straight, but some eventually come to a stop.
<A19> and <A17> decelerate and stop, while <A16> decelerates and then drives straight.
After initially decelerating, <A17> stops and then remains still.
<A20> parks towards the end of the scenario, from 65 seconds onwards.
The simulation features a mix of accelerating, decelerating, turning, and stopping vehicles, as well as parked ones.
After 40 seconds, <A26> accelerates and turns left.
Make <A28> and <A16> drive straight after 15 seconds into the simulation.
<A27>, <A25>, <A26>, <A4>, and <A5> all make left turns at the beginning of the simulati

### Check Motion Tags

In [5]:
v_action_str = batch.extras['condition']['v_action_tag']['caption_str']
motion_tags = v_action_str.split(', ')

print('\n'.join(motion_tags))

LeftTurn(<A26>: 0-45)
Straight(<A11>: 0-80)
Stopping(<A23>: 0-50)
Straight(<A19>: 0-55)
Parked(<A23>: 50-75)
Stopping(<A11>: 10-80)
Straight(<A21>: 70-80)
Decelerate(<A23>: 0-15)
Straight(<A16>: 0-80)
Stopping(<A19>: 0-55)
Stopping(<A10>: 0-30)
LeftTurn(<A4>: 0-80)
Accelerate(<A3>: 70-80)
Straight(<A5>: 75-80)
Straight(<A17>: 0-70)
Straight(<A15>: 0-35)
Decelerate(<A11>: 15-30)
Straight(<A28>: 0-40)
Stopping(<A17>: 0-70)
Parked(<A6>: 0-65)
Straight(<A22>: 0-30)
Parked(<A19>: 55-80)
Parked(<A2>: 0-40)
Parked(<A15>: 35-75)
Parked(<A1>: 0-45)
LeftTurn(<A5>: 0-75)
Accelerate(<A26>: 0-40)
Parked(<A24>: 0-65)
Straight(<A3>: 70-80)
Parked(<A29>: 0-35)
Accelerate(<A4>: 55-65)
Stopping(<A22>: 0-30)
Straight(<A0>: 0-75)
Accelerate(<A1>: 45-80)
Straight(<A12>: 50-80)
Straight(<A1>: 45-80)
Parked(<A20>: 5-80)
Stopping(<A0>: 0-75)
Parked(<A8>: 35-80)
Decelerate(<A14>: 0-5)
LeftTurn(<A25>: 0-55)
Accelerate(<A12>: 50-80)
Parked(<A17>: 70-80)
Parked(<A12>: 0-50)
Accelerate(<A28>: 0-40)
Stopping(<A16>:

### Check Goal Points

In [6]:
batch.extras['condition']['goal']['input']

tensor([[[ 2.5534e+00, -3.9919e-02,  8.0000e+01],
         [ 4.8959e-01,  4.9266e-02,  8.0000e+01],
         [ 1.1864e+01,  3.4917e-02,  8.0000e+01],
         [ 7.5283e-02,  2.3851e-02,  8.0000e+01],
         [ 5.2751e+00, -1.5578e-02,  8.0000e+01],
         [ 6.9435e-01,  5.1188e-03,  8.0000e+01],
         [ 0.0000e+00,  0.0000e+00,  3.5000e+01],
         [ 1.5717e+01,  4.2830e-02,  8.0000e+01],
         [ 1.0735e+01, -8.5748e-02,  8.0000e+01],
         [ 5.0816e+00,  3.0533e-02,  8.0000e+01],
         [ 3.0420e-02,  7.5853e-02,  8.0000e+01],
         [-1.7363e-02,  1.5898e-02,  8.0000e+01],
         [ 5.2752e-01,  9.2453e-02,  8.0000e+01],
         [ 3.5831e+01,  5.5261e+01,  8.0000e+01],
         [ 2.7373e+00, -1.2831e-02,  8.0000e+01],
         [ 0.0000e+00,  0.0000e+00,  8.0000e+01],
         [ 5.1821e-02,  3.5511e-02,  8.0000e+01],
         [ 7.1862e+00, -5.3879e-03,  8.0000e+01],
         [ 2.1477e+01, -7.7476e-02,  8.0000e+01],
         [ 6.3424e+01,  4.0411e-02,  4.2000e+01],


In [7]:
v_action_str = batch.extras['condition']['v_action_tag']['caption_str']
motion_tags = v_action_str.split(', ')

print('\n'.join(motion_tags))

LeftTurn(<A26>: 0-45)
Straight(<A11>: 0-80)
Stopping(<A23>: 0-50)
Straight(<A19>: 0-55)
Parked(<A23>: 50-75)
Stopping(<A11>: 10-80)
Straight(<A21>: 70-80)
Decelerate(<A23>: 0-15)
Straight(<A16>: 0-80)
Stopping(<A19>: 0-55)
Stopping(<A10>: 0-30)
LeftTurn(<A4>: 0-80)
Accelerate(<A3>: 70-80)
Straight(<A5>: 75-80)
Straight(<A17>: 0-70)
Straight(<A15>: 0-35)
Decelerate(<A11>: 15-30)
Straight(<A28>: 0-40)
Stopping(<A17>: 0-70)
Parked(<A6>: 0-65)
Straight(<A22>: 0-30)
Parked(<A19>: 55-80)
Parked(<A2>: 0-40)
Parked(<A15>: 35-75)
Parked(<A1>: 0-45)
LeftTurn(<A5>: 0-75)
Accelerate(<A26>: 0-40)
Parked(<A24>: 0-65)
Straight(<A3>: 70-80)
Parked(<A29>: 0-35)
Accelerate(<A4>: 55-65)
Stopping(<A22>: 0-30)
Straight(<A0>: 0-75)
Accelerate(<A1>: 45-80)
Straight(<A12>: 50-80)
Straight(<A1>: 45-80)
Parked(<A20>: 5-80)
Stopping(<A0>: 0-75)
Parked(<A8>: 35-80)
Decelerate(<A14>: 0-5)
LeftTurn(<A25>: 0-55)
Accelerate(<A12>: 50-80)
Parked(<A17>: 70-80)
Parked(<A12>: 0-50)
Accelerate(<A28>: 0-40)
Stopping(<A16>:

### Check Route Sketch

In [8]:
batch.extras['condition']['drag_point']['input']


tensor([[[[        nan,         nan],
          [        nan,         nan],
          [        nan,         nan],
          [        nan,         nan],
          [        nan,         nan],
          [        nan,         nan],
          [        nan,         nan],
          [ 1.7057e+01, -7.8410e-02],
          [ 1.7395e+01, -2.0600e-01],
          [ 1.7269e+01, -2.1200e-01],
          [ 1.7278e+01, -1.9441e-01],
          [ 1.7439e+01, -1.5864e-01],
          [ 1.7534e+01, -7.5896e-02],
          [ 1.7433e+01, -2.5926e-01],
          [        nan,         nan],
          [        nan,         nan]],

         [[        nan,         nan],
          [        nan,         nan],
          [-1.3289e-01,  1.8029e-01],
          [-3.2510e-03,  1.2322e-01],
          [-5.1497e-02,  1.0570e-02],
          [ 2.0104e-03,  8.4296e-02],
          [-9.3645e-02, -6.5237e-02],
          [-6.2406e-02, -4.7273e-03],
          [ 9.5079e-03,  2.1235e-02],
          [        nan,         nan],
          